In [1]:
!pip install tensorflow-gpu

     |████████████████████████████████| 458.3 MB 13 kB/s 


In [2]:
!nvidia-smi

Sat Sep 25 11:24:14 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import tensorflow as tf
tf.__version__

'2.6.0'

In [4]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

In [5]:
#resize all images 
IMAGE_SIZE = [224,224]

train_path = '/content/drive/MyDrive/tomatoset/train'
valid_path = '/content/drive/MyDrive/tomatoset/valid'

In [6]:
# Inception v3
inception = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

87924736/87910968 [==============================] - 1s 0us/step


In [7]:
# ignoring training from existing weights
for layer in inception.layers:
    layer.trainable = False

In [9]:
# getting the labels
folders = glob('/content/drive/MyDrive/tomatoset/train/*')
folders

['/content/drive/MyDrive/tomatoset/train/Tomato___Bacterial_spot',
 '/content/drive/MyDrive/tomatoset/train/Tomato___Early_blight',
 '/content/drive/MyDrive/tomatoset/train/Tomato___healthy',
 '/content/drive/MyDrive/tomatoset/train/Tomato___Tomato_mosaic_virus',
 '/content/drive/MyDrive/tomatoset/train/Tomato___Septoria_leaf_spot',
 '/content/drive/MyDrive/tomatoset/train/Tomato___Target_Spot',
 '/content/drive/MyDrive/tomatoset/train/Tomato___Leaf_Mold',
 '/content/drive/MyDrive/tomatoset/train/Tomato___Spider_mites Two-spotted_spider_mite',
 '/content/drive/MyDrive/tomatoset/train/Tomato___Tomato_Yellow_Leaf_Curl_Virus',
 '/content/drive/MyDrive/tomatoset/train/Tomato___Late_blight']

In [12]:
# layers
x = Flatten()(inception.output)
prediction = Dense(len(folders), activation='softmax')(x)


In [13]:
# creating  model
model = Model(inputs=inception.input, outputs=prediction)

# structure model
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [14]:
#cost and optimization
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [25]:
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/tomatoset/train',
                                                 target_size = (224, 224),
                                                 batch_size = 16,
                                                 class_mode = 'categorical')

Found 11010 images belonging to 10 classes.


In [ ]:
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/tomatoset/valid',
                                            target_size = (224, 224),
                                            batch_size = 16,
                                            class_mode = 'categorical')

In [ ]:
#train model 
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

In [ ]:
from tensorflow.keras.models import load_model

model.save('detector.tflite')